![alt text](https://drive.google.com/uc?export=view&id=1DXUVHxd4t15mfuqMgMCLnsP4jWVI5EWz)

---
<br>
© 2024 Copyright The University of New South Wales - CRICOS 00098G

**Author**: Oscar Perez-Concha: o.perezconcha@unsw.edu.au

**Contributors/Co-authors**: Marta Fredes-Torres, Zhisheng (Sandy) Sa and Matthew Sainsbury-Dale.








---




# Laboratory 3: Model Evaluation and Improvement.
# Cross-Validation for hyper-parameter tuning using `scikit-learn`'s function `GridSearchCV`.


Goal/Research question:To develop a predictive algorithm for forecasting hospital readmissions within 30 days post-discharge; this exercise will specifically focus on tuning the hyper-parameter C (sometimes called $\alpha$ or $λ$, where $\alpha =  C = \frac {1}{\lambda}$).
We will apply this tuning to our predictive algorithm, Lasso.

![alt text](https://drive.google.com/uc?export=view&id=105SGqeyo8RgLhSO8mN7ZE5OsG0YiLPKt)



---





---



# 1. Aims of the Exercise:

1. To become familiar with using a validation set to find the best hyper-parameters for a model. Remember that the hyper-parameters are defined by the user, whereas parameters (e.g., the beta coefficients in logistic regression) are found automatically by fitting the model.
2. To become familiar with a grid search: the commonly used method for tuning hyper-parameters is via a grid search, which entails testing many combinations of the hyper-parameters of interest.
3. To become familiar with k-fold cross-validation (k-CV) and grid search.
4. To become familiar with Python pipelines.

It aligns with these learning outcomes of our course:

1.	Distinguish a range of task specific machine learning techniques appropriate for Health Data Science.
2.	Design machine learning tasks for Health Data Science scenarios.




Follow the instructions given and if you have any questions, please use the **Comment section** in **Open Learning**.



---





---



# 2. Initial Docstring:

All programs should have an initial docstring comment. It must include at least the following elements:

* Purpose: what is the aim of your code?
* Date created
* Author
* Date modified
* Author of the modification
* Method: how did you go about solving the problem?
* Data dictionary: The data dictionary should contain all the important variables and constants defined, their datatype (float, string, int) and a short description of what they are.
* List and defintions of functions: similar to the data dictionary, but with functions.
* List of libraries: libraries used in the program and their functionality.

Is there anything else you think we should include in the docstring? Please comment in the comments section of this week's laboratory.

Please read these two documents:
1. pandas docstring guide: https://pandas.pydata.org/pandas-docs/version/0.23/contributing_docstring.html
2. Style guide: https://www.cse.unsw.edu.au/~en1811/resources/style.html


In addition to this initial docstring, please comment and document all the steps and rationale of your code. Use text cells or docstrings within your Python code.

<b> Docstring:</b>
#####################################################################################################################

(double-click here)


#####################################################################################################################



---





---



# 3. Dataset

In [1]:
# Insert your comments and explanations


import sys
import numpy as np
import pandas as pd


In [2]:
# Insert your comments and explanations


# Mount Google Drive
# We do not need to run this cell if you are not running this notebook in Google Colab

if 'google.colab' in str(get_ipython()):
    from google.colab import drive # import drive from Gogle colab
    root = '/content/drive'     # default location for the drive
    # print(root)                 # print content of ROOT (Optional)
    drive.mount(root)
else:
    print('Not running on CoLab')

Mounted at /content/drive


In [3]:
# Insert your comments and explanations

from pathlib import Path

if 'google.colab' in str(get_ipython()):
    # EDIT THE PROJECT PATH IF DIFFERENT WITH YOUR ONE
    # You may need to change 'MyDrive' to 'My Drive'.
    project_path = Path(root) / 'MyDrive' / 'Colab Notebooks'

    # OPTIONAL - set working directory according to your google drive project path
    # import os
    # Change directory to the location defined in project_path
    # os.chdir(project_path)
else:
    project_path = Path()

As we explained in the previous exercise, we already cleaned and created dummy variables for the diabetes dataset. In the Exercise 1 of Chapter 2 we saved this 'prepared' dataset using `pickle`. Now, we will load this data using 'pickle' ([more information](https://docs.python.org/3/library/pickle.html)). In this case, we will read it from last week's folder. You will have to change the path according to the folder where you stored your `hospital_data.pickle`.

In [4]:
# Insert your comments and explanations


pickle_data_path = Path(project_path) /'hospital_final.pickle'
hospital = pd.read_pickle(pickle_data_path)

In [5]:
# Insert your comments and explanations

# Santity checks - Please add many as you wish.
hospital.head(n=20)

,los,Age,sex,max_glu_serum,A1Cresult,number_diagnoses,num_lab_procedures,num_procedures,num_medications,number_emergency,...,group_name_1,group_name_2,group_name_3,readmission,admission_type_id_cat,discharge_disposition_id_cat,admission_source_id_cat,discharge_disposition_grouped,admission_source_grouped,admission_type_grouped
0,2,79,Female,None,None,9,38,0,12,0,...,Other,Endocrine,Infectious,no,3,18,7,nan,Emergency Room,Elective
1,5,59,Male,None,>8,8,49,0,16,0,...,Endocrine,Cardiac_&_circulatory,Other,no,3,1,7,nan,Emergency Room,Elective
2,2,33,Female,None,None,5,62,0,15,1,...,Other,Endocrine,Cardiac_&_circulatory,no,3,1,7,nan,Emergency Room,Elective
3,6,42,Female,None,None,9,77,0,30,0,...,Infectious,Respiratory,Endocrine,no,3,6,7,nan,Emergency Room,Elective
4,1,62,Male,None,None,7,13,5,6,0,...,Cardiac_&_circulatory,Cardiac_&_circulatory,Cardiac_&_circulatory,no,3,1,7,nan,Emergency Room,Elective
5,1,53,Female,None,None,9,11,3,16,0,...,Cardiac_&_circulatory,Respiratory,Cardiac_&_circulatory,no,3,1,7,nan,Emergency Room,Elective
6,10,80,Female,None,None,9,56,3,13,0,...,Cardiac_&_circulatory,Other,Cardiac_&_circulatory,no,3,3,7,nan,Emergency Room,Elective
7,2,44,Male,None,None,3,1,2,22,0,...,Other,Endocrine,Other,no,3,18,7,nan,Emergency Room,Elective
8,1,73,Male,None,None,9,9,0,5,0,...,Cardiac_&_circulatory,Endocrine,Other,no,3,7,7,nan,Emergency Room,Elective
9,4,64,Female,None,None,4,19,4,24,0,...,Other,Cardiac_&_circulatory,Endocrine,no,3,18,7,nan,Emergency Room,Elective




---



\

# 4. Hyper-parameter tuning: Grid Search with Cross-Validation (GridSearchCV)



---



Remember that:


1. Cross-validation is just a method to better estimate the performance of a model.
2. Cross-validation can be used either:


a. To just compute the performance of a model using different partitions of training and test sets and/or different hyper-parameters.

b. To compute the performance of a model using different partitions of training and test sets and/or different hyper-parameters AND to select the best resulting model out of that process. We will see this second option in this exercise.


---



The most commonly used method for tuning hyper-parameters is via a grid search, which entails testing many combinations of the hyper-parameters of interest.<p>
    
We want to combine the benefits of cross-validation with the grid search. We will seek to find the model with the best accuracy by using cross-validation. We will use the `sklearn.model_selection.GridSearchCV` function from sklearn ([More information](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html))
    


---

We have one hyper-parameter that we would like to tune:
C, ($C=\alpha=1/\lambda$)
<p>

---

Let's say we want to try:
C = 0.001, 0.01, 0.1, 1, 10, 100.

---

Please note that we can tune multiple hyper-parameters, including class weight, C, penalty, among others. In this exercise, we will demonstrate how to use grid search to tune only one parameter, 'C'. In addition, we will choose **L1 regularisation (Lasso) for this problem**. Feel free to add other hyper-parameters and evaluate if the model improves.



---



## 4.1 Splitting the feature variables from the response

In this step, we will also convert responses to binary for later steps.

In [6]:
# Insert your comments and explanations

# Dividing my dataset in X and y (outcome)
X = hospital.drop(axis=1, columns=['readmission'])
y = hospital['readmission']

In [7]:
# Insert your comments and explanations

# Sanity check
X.head(n=20)

,los,Age,sex,max_glu_serum,A1Cresult,number_diagnoses,num_lab_procedures,num_procedures,num_medications,number_emergency,...,number_outpatient,group_name_1,group_name_2,group_name_3,admission_type_id_cat,discharge_disposition_id_cat,admission_source_id_cat,discharge_disposition_grouped,admission_source_grouped,admission_type_grouped
0,2,79,Female,None,None,9,38,0,12,0,...,0,Other,Endocrine,Infectious,3,18,7,nan,Emergency Room,Elective
1,5,59,Male,None,>8,8,49,0,16,0,...,0,Endocrine,Cardiac_&_circulatory,Other,3,1,7,nan,Emergency Room,Elective
2,2,33,Female,None,None,5,62,0,15,1,...,1,Other,Endocrine,Cardiac_&_circulatory,3,1,7,nan,Emergency Room,Elective
3,6,42,Female,None,None,9,77,0,30,0,...,0,Infectious,Respiratory,Endocrine,3,6,7,nan,Emergency Room,Elective
4,1,62,Male,None,None,7,13,5,6,0,...,0,Cardiac_&_circulatory,Cardiac_&_circulatory,Cardiac_&_circulatory,3,1,7,nan,Emergency Room,Elective
5,1,53,Female,None,None,9,11,3,16,0,...,0,Cardiac_&_circulatory,Respiratory,Cardiac_&_circulatory,3,1,7,nan,Emergency Room,Elective
6,10,80,Female,None,None,9,56,3,13,0,...,0,Cardiac_&_circulatory,Other,Cardiac_&_circulatory,3,3,7,nan,Emergency Room,Elective
7,2,44,Male,None,None,3,1,2,22,0,...,0,Other,Endocrine,Other,3,18,7,nan,Emergency Room,Elective
8,1,73,Male,None,None,9,9,0,5,0,...,0,Cardiac_&_circulatory,Endocrine,Other,3,7,7,nan,Emergency Room,Elective
9,4,64,Female,None,None,4,19,4,24,0,...,0,Other,Cardiac_&_circulatory,Endocrine,3,18,7,nan,Emergency Room,Elective


In [8]:
# Insert your comments and explanations

# Sanity check
X.shape

(69267, 21)

In [9]:
# Insert your comments and explanations

# Sanity Checks:
print('******************************************')
print('y - Yes values =', sum(i =='yes' for i in y))
print('y - No values =', sum(i =='no' for i in y))
print('******************************************\n')

# Create y_binary
y_binary = [0 if x=='no' else 1 for x in y]

# Sanity Checks:
print('\n******************************************')
print('y_binary - 1 values =', sum(i ==1 for i in y_binary))
print('y_binary - 0 values =', sum(i ==0 for i in y_binary))
print('******************************************')

******************************************
y - Yes values = 11919
y - No values = 57348
******************************************


******************************************
y_binary - 1 values = 11919
y_binary - 0 values = 57348
******************************************




---



## 4.2  Split the whole dataset into a train and a test set (20% of the total)
Now let's split the data into a training and test set. We will include the optional argument `stratify = y` to preserve the ratio between readmission = yes to readmission = no.

In [10]:
# Insert your comments and explanations

from sklearn.model_selection import train_test_split
# Split X and y into 80% train and 20% test data (roughly), set random state for reproducability and stratify responses
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=0, stratify = y)



---



## 4.3 Define the Pipeline

### <font color='blue'> Question: Define the scaler we will use, and the estimator. As before, choose the scaler ("Transform") to be StandardScaler(), and the estimator ("Estimator") to be L1 Logistic Regression. </font>

To read about Pipelines:
1. Sklearn's API: [sckit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html). Check the example for pipelines in the API. Instead of SVC(), we are using LogisticRegression().
2. Textbook.




In [11]:
# Insert your comments and explanations

# Write Python Code Here:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Scaler object
standard_scaler = StandardScaler()

# Logistic Regression object
log_reg = LogisticRegression(penalty='l1', solver='liblinear')

# Pipeline object
pipeln = Pipeline([('Transform', standard_scaler), ('Estimator', log_reg)])



---



## 4.4 Define the hyper-parameters grid

### <font color='blue'> Question: Define the hyper-parameter grid. </font>

In [15]:
# Insert your comments and explanations

# Write Python Code Here:
param_Estimator__C = [0.001, 0.01, 0.1, 1, 10, 100]
param_grid = {'Estimator__C': param_Estimator__C}

# sanity check
print("Parameter Grid:")
print("C: {}". format(param_grid['Estimator__C']))


Parameter Grid:
C: [0.001, 0.01, 0.1, 1, 10, 100]




---



Remember that we are going to use a pipeline. In this case, there are two processes that are executed for each iteration of the cross-validation for hyper-parameter tuning:

First, the standardization of the features.
1. Then, the fitting of the logistic model.
2. This means that we have to indicate which of these processes our hyper-parameter grid belongs to.

If you have named your logistic regression model 'Estimator', you can designate its hyper-parameters by naming the parameter grid "Estimator__'parameter_name'". For example, we tell the computer that C is meant for the logistic regression estimator by defining it as 'Estimator__C' in the param_grid.

### <font color='blue'> Question: Initialise the GridSearchCV class by passing it the pipeline we have created, our paramater grid, the score you would like to use to select the best model, and specifying the number of folds. We must consider the computational complexity of the algorithm, so we can't set cv too high. In this case, let's choose 3 folds.</font>

The default score for the `GridSearchCV` we defined above is **accuracy**.

Different scoring measures can be used in the grid search. This can be a simple string or you can pass a list of values to use as multiple evaluation metrics, with the results for each metric viewed through cv_results_. If multiple scoring metrics are set then the refit parameter needs to be set to tell the function which scorer will be used to evaluate the best parameter settings.

For example, we could extract the F1-score of our GridSearchCV by using the command: **`GridSearchCV(pipeline, param_grid,...,scoring='f1')`**

[More information](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html?highlight=cross_validate#sklearn.model_selection.cross_validate)

In [20]:
# Insert your comments and explanations

# Write Python Code Here:
# Import the grid search CV class
from sklearn.model_selection import GridSearchCV

# Create the grid search object
#grid_search = GridSearchCV(pipeln, param_grid, scoring='accuracy', cv=3)
grid_search = GridSearchCV(pipeln, param_grid, scoring='f1', cv=3)



---



## 4.5 Find the best parameters

### <font color='blue'> Question: Now we train the grid_search object. Note that grid_search behaves similarly to other classifiers, in the sense that we can use the methods fit, predict and score with it. When we use fit, it performs the grid cross-validation we designed during its initialisation. </font>

**Please note that the following code takes a while to run**:

In [24]:
# Insert your comments and explanations

# Write Python Code Here:
print("Beast parameters: {}". format(grid_search.best_params_))
print("Best score: {:.4f}". format(grid_search.best_score_))

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

**To have greater clarity of our results, we proceed to analyse the heatmap.**

Let's visualise the cross-validation f1-scores as a function of the parameter 'C'.

In [23]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings; warnings.simplefilter('ignore') #prevent warnings

# Creating DataFrame with GridSearchCV results. df stands for Dataframe
df_gridsearch = pd.DataFrame(grid_search.cv_results_)

max_scores = df_gridsearch.groupby(['param_Estimator__C']).max()
max_scores = max_scores.unstack()[['mean_test_score']]
max_scores.fillna(value=np.nan, inplace=True)
max_scores = max_scores.apply(pd.to_numeric)
sns.heatmap(max_scores.mean_test_score.to_frame(), annot=True, fmt='.5g',xticklabels="")
plt.xlabel("score")

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'



---



## 4.6 Evaluate on the performance of the resulting model

<font color=red> **Very very important:** Recall that to this point we have not used the test set</font> - only the training set was used for tuning the hyper-parameters.</font>

![alt text](https://drive.google.com/uc?export=view&id=105SGqeyo8RgLhSO8mN7ZE5OsG0YiLPKt)

**Very very important:** "Fitting the GridSearchCV object not only searches for the best parameters, but also
automatically fits a new model on the whole training dataset with the parameters that
yielded the best cross-validation performance".

### <font color='blue'> Question: Calculate confusion matrix, and classification report for the training and test sets. Comment on the results below </font>
<font color='green'> Tip: read [GridSerchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) </font>

<font color='green'>Do we need to scale the test set? Yes, we do, but we do not have to do much. When we apply `.predict` on new data using  our `grid_search(pipeline,...)` object, all the steps defined in the `pipeline` will be applied automatically, including the scaling, which was the first step. Remember our `pipeline` was defined as `pipeln = Pipeline([('Transform', standard_scaler), ('Estimator', log_reg)])`.

In [ ]:
# Insert your comments and explanations

# Write Python Code Here:

<b> Write your thoughts here:</b>
#####################################################################################################################


#####################################################################################################################